In [2]:
import pandas as pd

In [33]:
class machine():
    
    def __init__(self, p, inp):
        self.p = p + [0] * 1000
        self.inputs = inp if type(inp)==list else [inp]
    
    def add_input(self, inp):
        self.inputs.append(inp)
    
    def run(self):
        p = self.p
        rel_base = 0
        def get_param(ip, modes, n):
            addr = p[ip+n]
            if modes[n-1]==0: return p[addr]              # position mode
            elif modes[n-1]==1: return addr               # immediate mode
            elif modes[n-1]==2: return p[rel_base + addr] # relative mode
            else:
                print("Unknown relative mode for read")
                return
        def set_param(ip, modes, n, val):
            addr = p[ip+n]
            if modes[n-1]==0: p[addr] = val
            elif modes[n-1]==2: p[rel_base + addr] = val
            else:
                print("Unknown relative mode for write")
        ip = 0
        while True:
            first = p[ip]
            op = first % 100
            if op == 99: return
            modes = list(map(int, str(first // 100)))
            if op in [1,2,7,8]: param_count = 3
            elif op in [5,6]: param_count = 2
            elif op in [3,4,9]: param_count = 1
            else:
                print("Unknown opcode", op)
                return
            while len(modes) < param_count: modes = [0] + modes
            modes = modes[::-1]
            if op == 3:
                set_param(ip, modes, 1, self.inputs[0])
                self.inputs = self.inputs[1:]
            elif op == 4: yield get_param(ip, modes, 1)
            elif op == 9: rel_base += get_param(ip, modes, 1)
            elif op == 1: set_param(ip, modes, 3, get_param(ip, modes, 1) + get_param(ip, modes, 2))
            elif op == 2: set_param(ip, modes, 3, get_param(ip, modes, 1) * get_param(ip, modes, 2))
            elif op == 5:
                if get_param(ip, modes, 1)!=0:
                    ip = get_param(ip, modes, 2)
                    continue
            elif op == 6:
                if get_param(ip, modes, 1)==0:
                    ip = get_param(ip, modes, 2)
                    continue
            elif op == 7:
                if get_param(ip, modes, 1) < get_param(ip, modes, 2): set_param(ip, modes, 3, 1)
                else: set_param(ip, modes, 3, 0)
            elif op == 8:
                if get_param(ip, modes, 1) == get_param(ip, modes, 2): set_param(ip, modes, 3, 1)
                else: set_param(ip, modes, 3, 0)
            ip += 1 + param_count     
    
    def start(self):
        return self.run()

In [4]:
with open('/Users/felixpuetsch/Downloads/input13.txt', 'rt') as f:
    prog = f.read()
    prog = list(map(int, prog.strip().split(',')))

In [127]:
from collections import defaultdict

def arcade(prog, inp=None):
    screen = defaultdict(int)
    m = machine(prog, inp)
    r = m.start()
    paddle, ball = None, None
    while True:
        try:
            x,y,t = next(r), next(r), next(r)
            screen[(x,y)] = t
            bricks = len(list(filter(lambda v:v[1]==2, screen.items())))
            if t==3: #paddle
                paddle = x
                #print(f"paddle = {paddle}")
            if t==4: #ball
                ball = x
                if not paddle: continue
                if paddle < ball: joy = 1
                elif paddle > ball: joy = -1
                else: joy = 0
                m.add_input(joy)
                #print(f"ball = {ball} joy = {joy} score = {screen[(-1,0)]}")
            if paddle and ball and bricks == 0: m.add_input(0)
        except StopIteration: break
        except IndexError: break
    return screen

def display(screen):
    tiles = ' ▓░=o'

    for y in range(22,0,-1):
        s = ""
        for x in range(35):
            c = tiles[ screen[(x,y)] ]
            s+=c
        print(s)

    print(screen[(-1,0)])

In [128]:
screen = arcade(prog)
display(screen)

▓                                 ▓
▓                =                ▓
▓                                 ▓
▓                                 ▓
▓              o                  ▓
▓                                 ▓
▓ ░░ ░░░ ░ ░   ░░░░ ░░ ░  ░ ░ ░░░ ▓
▓ ░  ░░     ░  ░░░░░ ░░ ░  ░ ░ ░░ ▓
▓ ░░░ ░░░░░  ░   ░ ░  ░░ ░░░░░  ░ ▓
▓  ░░  ░░░ ░░ ░░░░░    ░ ░ ░ ░░░  ▓
▓ ░     ░░░  ░ ░░ ░░ ░ ░░░░░░░ ░  ▓
▓  ░░   ░ ░░░  ░░░░░  ░░ ░░   ░░  ▓
▓    ░░  ░░   ░░░░ ░ ░░░ ░      ░ ▓
▓ ░  ░░ ░░░ ░░  ░    ░░ ░░░░░ ░░  ▓
▓ ░░░░    ░░░ ░░  ░░ ░░ ░ ░░ ░░ ░ ▓
▓ ░     ░ ░ ░ ░  ░░░   ░     ░░░░ ▓
▓ ░ ░░ ░░    ░ ░░░ ░  ░░░    ░░░  ▓
▓ ░ ░░░░   ░░  ░░░ ░░░░ ░░  ░ ░   ▓
▓  ░░░░ ░ ░░░░░   ░ ░░ ░  ░░ ░░   ▓
▓  ░░   ░░░   ░░░ ░ ░ ░ ░ ░ ░░░░░ ▓
▓ ░ ░   ░░  ░ ░░░ ░░  ░░  ░░░░░░░ ▓
▓                                 ▓
0


In [129]:
prog2 = prog.copy()
prog2[0] = 2
screen = arcade(prog2, 0)
display(screen)

▓                                 ▓
▓                  =              ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                  o              ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
▓                                 ▓
12952
